In [18]:
import pandas as pd
import ktrain
from ktrain import text
import numpy as np

In [3]:
df=pd.read_csv("webis_train.csv")
df2=pd.read_csv("webis_test.csv")

## Handeling dataset

In [19]:
input=df["postText"].fillna("")
input=input.values.tolist()
output=df["truthClass"]
output=output.values.tolist()
input2=df2["postText"].fillna("")
input2=input2.values.tolist()
output2=df2["truthClass"]
output2=output2.values.tolist()
#indx = df["truthClass"].str.len().idxmax()
#len(df["truthClass"][indx])
classname=['clickbait','no-clickbait']
classname=np.array(classname)

In [5]:
model_name="distilbert-base-uncased"
classifier= text.Transformer(model_name,maxlen=512,class_names=classname)

In [6]:
train_data=classifier.preprocess_train(input,output)
test_data=classifier.preprocess_train(input2,output2)

preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 18
	99percentile : 21


d:\newtral task\VE\lib\site-packages\ktrain\utils.py:625: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 18
	99percentile : 20


Is Multi-Label? False


In [7]:
model=classifier.get_classifier()

In [14]:
learner= ktrain.get_learner(model,train_data=train_data,val_data=test_data,batch_size=8)

In [15]:
learner.fit_onecycle(1e-3,3)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/3
2443/2443 [==============================] - ETA: 0s - loss: 0.5456 - accuracy: 0.7610 

KeyboardInterrupt: 

## Save and Validate

In [20]:
learner.validate(classname)

ValueError: data must be tuple of numpy.ndarrays or BERT-style tuple or an instance of Iterator

In [21]:
Predictor= ktrain.get_predictor(learner.model,preproc=classifier)

In [22]:
Predictor.predict("put the tweet here")

'no-clickbait'

In [23]:
learner.save_model("our tuned model")